The first thing we will need to do is load the tensorflow package. "tf" is usually used. 

In [1]:
import tensorflow as tf

Tensorflow has its own versions of typical objects (floats, ints, booleans, etc.). The most important ones are (of course) tensors, which mostly function similar to numpy arrays. They come in two types, "tf.constant" and "tf.Variable". They have an N1 x N2 x N3... type shape, represented as [N1,N2,N3,..]. No triangular tensors are permitted, for example. Sparse tensors exist, but they're not particularly well supported. By default, everything is float32. 

In [2]:
constant_tensor = tf.constant([1.,2.,3.])
variable_tensor = tf.Variable([[1.,2.],[3.,4.]])
float64_tensor = tf.constant([2.,4.,5.],dtype="float64")

We access the shape with .shape

In [3]:
constant_tensor.shape

TensorShape([3])

In [4]:
variable_tensor.shape

TensorShape([2, 2])

When tensors have the same shape, multiplication and addition work componentwise. 

In [5]:
x1 = tf.constant([[1.,2.],[3.,4.]])
x2 = tf.constant([[3.,4.],[5.,1.]])

x1+x2

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4., 6.],
       [8., 5.]], dtype=float32)>

In [6]:
x1*x2

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 3.,  8.],
       [15.,  4.]], dtype=float32)>

When adding/multiply tensors with different shapes, it will try to “broadcast”, by extending one by repetition to make it the same shape as the other. This only works if broadcasting can make them compatible, otherwise it throws up an error

In [7]:
x1 = tf. constant ([[1. ,2.] ,[5. ,6.]])
x2 = tf. constant ([[0.] ,[1.]])

#x3 is the "broadcasted" version of x2
x3 = tf. constant ([[0. ,0.] ,[1. ,1.]])

x1+x2

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [6., 7.]], dtype=float32)>

In [8]:
x1+x3

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 2.],
       [6., 7.]], dtype=float32)>

Single-variable functions act componentwise, and are mostly the same as numpy, using tf.math. instead of np.

In [9]:
x1 = tf.constant ([[1. ,2.,3.] ,[4.,5. ,6.]])
tf.math.exp (x1)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  2.7182817,   7.389056 ,  20.085537 ],
       [ 54.59815  , 148.41316  , 403.4288   ]], dtype=float32)>

In [10]:
tf.math.sqrt (x1)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.       , 1.4142135, 1.7320508],
       [2.       , 2.236068 , 2.4494898]], dtype=float32)>

A few very useful, tensor-based commands are the following. We can add all elements, 

In [11]:
tf.reduce_sum(x1)

<tf.Tensor: shape=(), dtype=float32, numpy=21.0>

reduce_sum can also sum over a particular axis. The axes are [0,1,...], in the same order as the shape [n1,n2,...]. Our tensor is [2,3], so setting axis=0 sums over the first, leaving a shape [3] tensor. 

In [12]:
tf.reduce_sum(x1,axis=0)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([5., 7., 9.], dtype=float32)>

tf.reduce_mean does the same, but with averages

In [13]:
tf.reduce_mean(x1,axis=1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

A fundamental operation is taking the derivative of a tensor with respect to another. This happens both when defining the loss function itself (in PINNs-type approaches) and when finding gradients for optimisation. We use autodiff, which has two modes (forward and backwards). I'll focus on forward. To take the derivative, you have to record the function itself in a tf.GradientTape environment.

In [14]:
x2 = tf.constant([2.,4.,6])

#Create the environment
with tf.GradientTape() as t1:
    
    #Tell it to "watch" the variable with respect to which you want to take the derivative
    t1.watch(x2)
    
    #Calculate the object of interest, this is y = x2[0]^2+x2[1]^2+x2[3]^2
    y = tf.reduce_sum(x2**2)

#Then take the derivative of y wrt x2, it should be 2*x2
dy = t1.gradient(y,x2)
print(dy)

tf.Tensor([ 4.  8. 12.], shape=(3,), dtype=float32)


There are two main differences between constants and variables, one is how they behave in a GradientTape. Variables are automatically watched, constants are not. This is what happens if it doesn't "watch"

In [15]:
#Create the environment
with tf.GradientTape() as t1:
    
    #Calculate the object of interest, this is y = x2[0]^2+x2[1]^2+x2[3]^2
    y = tf.reduce_sum(x2**2)

#Then take the derivative of y wrt x2
dy = t1.gradient(y,x2)
print(dy)

None


Now we try the same with a Variable

In [16]:
x2 = tf.Variable([2.,4.,6.])

#Create the environment
with tf.GradientTape() as t1:
    
    #Calculate the object of interest, this is y = x2[0]^2+x2[1]^2+x2[3]^2
    y = tf.reduce_sum(x2**2)

#Then take the derivative of y wrt x2
dy = t1.gradient(y,x2)
print(dy)

tf.Tensor([ 4.  8. 12.], shape=(3,), dtype=float32)


If the output and input tensors have the same shape, then gradient works componentwise. 

In [17]:
with tf.GradientTape() as t1:
    
    #y is now x2**2 componentwise
    y = x2**2
    print("y=", y)

dy = t1.gradient(y,x2)
print("grad = ", dy)

y= tf.Tensor([ 4. 16. 36.], shape=(3,), dtype=float32)
grad =  tf.Tensor([ 4.  8. 12.], shape=(3,), dtype=float32)


We can do higher order derivatives with nested GradientTapes. We will show how to do this with the Jacobian of the gradient (The Hessian). The command jacobian does what you would expect - with N dimensional input and output, it returns the NxN matrix of each output wrt each input

In [18]:
x1 = tf.Variable([2.,3.])

with tf.GradientTape() as t1:
    with tf.GradientTape() as t2:
        y = tf.reduce_sum(x1**2)/2
    dy = t2.gradient(y,x1)
Hy = t1.jacobian(dy,x1)
print(Hy)

tf.Tensor(
[[1. 0.]
 [0. 1.]], shape=(2, 2), dtype=float32)


If we have tensorial input -> scalar output, and we do computations componentwise, gradient works componentwise as well. Let's see an example

In [19]:
xy = tf.constant([[1.,2.],[3.,4.],[5.,6.]])

with tf.GradientTape() as t1:
    t1.watch(xy)

    #We will do the sums each element of xy^2. 

    z = tf.reduce_sum(xy**2,axis=1)
    print("z=", z)

dz = t1.gradient(z,xy)
print("dz = ", dz)

z= tf.Tensor([ 5. 25. 61.], shape=(3,), dtype=float32)
dz =  tf.Tensor(
[[ 2.  4.]
 [ 6.  8.]
 [10. 12.]], shape=(3, 2), dtype=float32)


The more "complicated" case is vectorial->vectorial, but still componentwise. We can have in mind a tensor of shape [N,3] mapping to one of shape [N,2], where "N" is the number of "copies" we want to consider. Our answer should be [N,2,3]. For this, we use batch_jacobian. I will use slicing to obtain the coordinates, and glue them together with "stack"

In [20]:
xyz = tf.constant([[1.,2.,3.],[4.,5.,6.],[7.,8.,9.],[10.,11.,12.]])

with tf.GradientTape() as t1:
    t1.watch(xyz)
    x = xyz[:,0]
    y = xyz[:,1]
    z = xyz[:,2]
    w1 = x*y
    w2 = y*z
    w = tf.stack([w1,w2],axis=-1)
    print(w)

jac_w = t1.batch_jacobian(w,xyz)
print("Jacobian:", jac_w)

tf.Tensor(
[[  2.   6.]
 [ 20.  30.]
 [ 56.  72.]
 [110. 132.]], shape=(4, 2), dtype=float32)
Jacobian: tf.Tensor(
[[[ 2.  1.  0.]
  [ 0.  3.  2.]]

 [[ 5.  4.  0.]
  [ 0.  6.  5.]]

 [[ 8.  7.  0.]
  [ 0.  9.  8.]]

 [[11. 10.  0.]
  [ 0. 12. 11.]]], shape=(4, 2, 3), dtype=float32)


One potent tool in tensorflow is graph mode. By default, everything runs in "Eager execution" - typical line-by-line execution of code. The wrapper @tf.function turns it into "Graph mode", which is a more optimised version of the function. If the inputs stay the same shape, repeatedly running the same function is much faster (Iterative methods!!). Let's see how this works. 

In [22]:
import time

def Eager_function(x):
    return tf.math.sin(x**2)

@tf.function
def Graph_function(x):
    return tf.math.sin(x**2)

x = tf.random.uniform([10**8])
t0 = time.time()
y = Eager_function(x)
print("Eager:",time.time()-t0)


x = tf.random.uniform([10**8])
t0 = time.time()
y = Graph_function(x)
print("Graph (First pass):",time.time()-t0)


x = tf.random.uniform([10**8])
t0 = time.time()
y = Graph_function(x)
print("Graph (Second pass):",time.time()-t0)


x = tf.random.uniform([10**8])
t0 = time.time()
y = Graph_function(x)
print("Graph (Third pass):",time.time()-t0)


Eager: 0.693925142288208
Graph (First pass): 0.13084030151367188
Graph (Second pass): 0.0565342903137207
Graph (Third pass): 0.05708456039428711
